In [1]:
import os
import json
import torch
import psycopg2
from transformers import AutoTokenizer, AutoModel
from IPython.display import clear_output
import numpy as np
from tqdm import tqdm
from umap import UMAP

# VETORIZANDO A EMENTA COMPLETA COM O BERTIMBAU LOCAL E SALVANDO NO BANCO

In [2]:
# Caminho local para o modelo BERTimbau (em formato PyTorch)
bert_model_path = "C:/Users/Loreane/Documents/bert"

# Define dispositivo (usa GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carrega tokenizer e modelo
tokenizer = AutoTokenizer.from_pretrained(bert_model_path)
model = AutoModel.from_pretrained(bert_model_path).to(device)
model.eval()

# Função para gerar embedding médio da última camada oculta
def gerar_vetor(texto: str):
    with torch.no_grad():
        inputs = tokenizer(
            texto,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        ).to(device)

        outputs = model(**inputs)
        vetor = torch.mean(outputs.last_hidden_state, dim=1)  # [1, hidden_size]
        return vetor.squeeze(0).cpu().numpy()  # (hidden_size,)

# Conexão com PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
)
cur = conn.cursor()

cur.execute("""
    SELECT id, ementa_completa
    FROM processos
    WHERE ementa_completa IS NOT NULL and vetor_ementa_completa IS NULL
""")
registros = cur.fetchall()
total = len(registros)

# Processa e atualiza os vetores
for i, (id_proc, ementa_completa) in enumerate(registros, start=1):
    vetor = gerar_vetor(ementa_completa)
    vetor_json = json.dumps(vetor.tolist())

    cur.execute("""
        UPDATE processos
           SET vetor_ementa_completa = %s
         WHERE id = %s
    """, (vetor_json, id_proc))

    conn.commit()  # commit a cada registro

    clear_output(wait=True)
    print(f"✅ Processo {i} de {total} atualizado (ID: {id_proc})")

cur.close()
conn.close()


Some weights of the model checkpoint at C:/Users/Loreane/Documents/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# VETORIZANDO A DESCRICAO + CASO E SALVANDO NO BANCO
Foi necessário refazer esta etapa pois no experimento fiz apenas pros processos que envolvem mei, e nesse teste quero avaliar todos processos, coloquei em campo separado do vetor no banco

In [3]:
# Caminho local para o modelo BERTimbau (em formato PyTorch)
bert_model_path = "C:/Users/Loreane/Documents/bert"

# Conexão com PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
)
cur = conn.cursor()

cur.execute("""
    SELECT id, descricao_caso, questoes_em_discussao
    FROM processos
    WHERE (descricao_caso is not null or questoes_em_discussao IS not null) and vetor_descricao_questao_testes is null 
""")
registros = cur.fetchall()
total = len(registros)

# Processa e atualiza os vetores
for i, (id_proc, desc, quest) in enumerate(registros, start=1):
    # concatena com separador
    texto = (desc or "").strip()
    if quest and quest.strip():
        texto = (texto + "\n\n" if texto else "") + quest.strip()
    print(texto)
    vetor = gerar_vetor(texto)
    vetor_json = json.dumps(vetor.tolist())

    cur.execute("""
        UPDATE processos
           SET vetor_descricao_questao_testes = %s
         WHERE id = %s
    """, (vetor_json, id_proc))

    conn.commit()  # commit a cada registro

    clear_output(wait=True)
    print(f"✅ Processo {i} de {total} atualizado (ID: {id_proc})")

cur.close()
conn.close()


# REDUZINDO A DIMENSIONALIDADE DO VETOR EMENTA COMPLETA PARA EXTRAÇÃO DE TÓPICOS (dimensionalidade 5) e salvando no banco

In [4]:
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
)
cur = conn.cursor()

print("🔍 Carregando embeddings do banco...")
cur.execute("""
    SELECT id, vetor_ementa_completa
      FROM processos
     WHERE vetor_ementa_completa IS NOT NULL
     AND vetor_ementa_reduzido_extracao_topico IS NULL
     ORDER BY id
""")
rows = cur.fetchall()

ids = [r[0] for r in rows]
embeddings = np.vstack([np.array(json.loads(r[1]), dtype=np.float32) for r in rows])
total = len(ids)
print(f"✅ {total} embeddings carregados")

if total == 0:
    cur.close()
    conn.close()
    print("Nada a processar. ✅")
else:
    # =========================
    # Ajusta UMAP (fit) e transforma em lotes de 500
    # =========================
    print("⚙️ Ajustando UMAP (fit único em toda a base)...")
    umap = UMAP(
        random_state=42,
        n_neighbors=30,
        n_components=5,
        min_dist=0.15,
        metric="euclidean"
    )
    umap.fit(embeddings)
    print("✅ UMAP ajustado")

    TRANSFORM_BATCH_SIZE = 100   # reduz de 500 em 500
    UPDATE_BATCH_SIZE = 100     # salva no banco em lotes (pode ser 500/1000/2000)

    reduced_cache = []  # acumula (embedding_reduzido, id) para updates
    reduzidos = 0       # contador de vetores já reduzidos
    salvos = 0          # contador de vetores já salvos no banco

    update_sql = """
        UPDATE processos
           SET vetor_ementa_reduzido_extracao_topico = %s
         WHERE id = %s
    """

    print("🚚 Iniciando redução em lotes de 500...")
    for start in range(0, total, TRANSFORM_BATCH_SIZE):
        end = min(start + TRANSFORM_BATCH_SIZE, total)
        X_batch = embeddings[start:end]
        Y_batch = umap.transform(X_batch)  # (batch, 5)

        # acumula para atualização no banco
        for y_vec, id_ in zip(Y_batch, ids[start:end]):
            reduced_cache.append((y_vec.tolist(), id_))

        reduzidos += (end - start)

        # feedback
        clear_output(wait=True)
        print(f"⚙️ Reduzindo dimensionalidade com UMAP...")
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")

        # escreve no banco quando atingir UPDATE_BATCH_SIZE (ou no final)
        if len(reduced_cache) >= UPDATE_BATCH_SIZE:
            cur.executemany(update_sql, reduced_cache)
            conn.commit()
            salvos += len(reduced_cache)
            reduced_cache.clear()

            clear_output(wait=True)
            print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
            print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    # salva o restante
    if reduced_cache:
        cur.executemany(update_sql, reduced_cache)
        conn.commit()
        salvos += len(reduced_cache)
        reduced_cache.clear()

        clear_output(wait=True)
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
        print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    print("🎉 Finalizado com sucesso!")
    cur.close()
    conn.close()

🔍 Carregando embeddings do banco...


ValueError: need at least one array to concatenate

## REDUZINDO A DIMENSIONALIDADE DO VETOR DESCRICAO + CASO PARA EXTRAÇÃO DE TÓPICOS (dimensionalidade 5) e salvando no banco

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
)
cur = conn.cursor()

print("🔍 Carregando embeddings do banco...")
cur.execute("""
    SELECT id, vetor_descricao_questao_testes
      FROM processos
     WHERE vetor_descricao_questao_testes IS NOT NULL
     AND vetor_descricao_questoes_testes_extracao IS NULL
     ORDER BY id
""") 
rows = cur.fetchall()

ids = [r[0] for r in rows]
embeddings = np.vstack([np.array(json.loads(r[1]), dtype=np.float32) for r in rows])
total = len(ids)
print(f"✅ {total} embeddings carregados")

if total == 0:
    cur.close()
    conn.close()
    print("Nada a processar. ✅")
else:
    # =========================
    # Ajusta UMAP (fit) e transforma em lotes de 500
    # =========================
    print("⚙️ Ajustando UMAP (fit único em toda a base)...")
    umap = UMAP(
        random_state=42,
        n_neighbors=30,
        n_components=5,
        min_dist=0.15,
        metric="euclidean"
    )
    umap.fit(embeddings)
    print("✅ UMAP ajustado")

    TRANSFORM_BATCH_SIZE = 100   # reduz de 500 em 500
    UPDATE_BATCH_SIZE = 100     # salva no banco em lotes (pode ser 500/1000/2000)

    reduced_cache = []  # acumula (embedding_reduzido, id) para updates
    reduzidos = 0       # contador de vetores já reduzidos
    salvos = 0          # contador de vetores já salvos no banco

    update_sql = """
        UPDATE processos
           SET vetor_descricao_questoes_testes_extracao = %s
         WHERE id = %s
    """

    print("🚚 Iniciando redução em lotes de 500...")
    for start in range(0, total, TRANSFORM_BATCH_SIZE):
        end = min(start + TRANSFORM_BATCH_SIZE, total)
        X_batch = embeddings[start:end]
        Y_batch = umap.transform(X_batch)  # (batch, 5)

        # acumula para atualização no banco
        for y_vec, id_ in zip(Y_batch, ids[start:end]):
            reduced_cache.append((y_vec.tolist(), id_))

        reduzidos += (end - start)

        # feedback
        clear_output(wait=True)
        print(f"⚙️ Reduzindo dimensionalidade com UMAP...")
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")

        # escreve no banco quando atingir UPDATE_BATCH_SIZE (ou no final)
        if len(reduced_cache) >= UPDATE_BATCH_SIZE:
            cur.executemany(update_sql, reduced_cache)
            conn.commit()
            salvos += len(reduced_cache)
            reduced_cache.clear()

            clear_output(wait=True)
            print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
            print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    # salva o restante
    if reduced_cache:
        cur.executemany(update_sql, reduced_cache)
        conn.commit()
        salvos += len(reduced_cache)
        reduced_cache.clear()

        clear_output(wait=True)
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
        print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    print("🎉 Finalizado com sucesso!")
    cur.close()
    conn.close()

# REDUZINDO A DIMENSIONALIDADE DO VETOR EMENTA COMPLETA PARA PLOTAGEM DE TÓPICOS (dimensionalidade 2) e salvando no banco

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
)
cur = conn.cursor()

print("🔍 Carregando embeddings do banco...")
cur.execute("""
    SELECT id, vetor_ementa_completa
      FROM processos
     WHERE vetor_ementa_completa IS NOT NULL
     AND vetor_ementa_reduzido_plotagem IS NULL
     ORDER BY id
""")
rows = cur.fetchall()

ids = [r[0] for r in rows]
embeddings = np.vstack([np.array(json.loads(r[1]), dtype=np.float32) for r in rows])
total = len(ids)
print(f"✅ {total} embeddings carregados")

if total == 0:
    cur.close()
    conn.close()
    print("Nada a processar. ✅")
else:
    # =========================
    # Ajusta UMAP (fit) e transforma em lotes de 500
    # =========================
    print("⚙️ Ajustando UMAP (fit único em toda a base)...")
    umap = UMAP(
        random_state=42,
        n_neighbors=15,
        n_components=2,
        min_dist=0.3,
        metric="cosine"
    ) 
    umap.fit(embeddings)
    print("✅ UMAP ajustado")

    TRANSFORM_BATCH_SIZE = 100   # reduz de 500 em 500
    UPDATE_BATCH_SIZE = 100     # salva no banco em lotes (pode ser 500/1000/2000)

    reduced_cache = []  # acumula (embedding_reduzido, id) para updates
    reduzidos = 0       # contador de vetores já reduzidos
    salvos = 0          # contador de vetores já salvos no banco

    update_sql = """
        UPDATE processos
           SET vetor_ementa_reduzido_plotagem = %s
         WHERE id = %s
    """

    print("🚚 Iniciando redução em lotes de 500...")
    for start in range(0, total, TRANSFORM_BATCH_SIZE):
        end = min(start + TRANSFORM_BATCH_SIZE, total)
        X_batch = embeddings[start:end]
        Y_batch = umap.transform(X_batch)  # (batch, 5)

        # acumula para atualização no banco
        for y_vec, id_ in zip(Y_batch, ids[start:end]):
            reduced_cache.append((y_vec.tolist(), id_))

        reduzidos += (end - start)

        # feedback
        clear_output(wait=True)
        print(f"⚙️ Reduzindo dimensionalidade com UMAP...")
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")

        # escreve no banco quando atingir UPDATE_BATCH_SIZE (ou no final)
        if len(reduced_cache) >= UPDATE_BATCH_SIZE:
            cur.executemany(update_sql, reduced_cache)
            conn.commit()
            salvos += len(reduced_cache)
            reduced_cache.clear()

            clear_output(wait=True)
            print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
            print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    # salva o restante
    if reduced_cache:
        cur.executemany(update_sql, reduced_cache)
        conn.commit()
        salvos += len(reduced_cache)
        reduced_cache.clear()

        clear_output(wait=True)
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
        print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    print("🎉 Finalizado com sucesso!")
    cur.close()
    conn.close()

In [25]:
# REDUZINDO A DIMENSIONALIDADE DO VETOR QUESTAO + DISCUSSAO PARA PLOTAGEM DE TÓPICOS (dimensionalidade 2) e salvando no banco

In [28]:
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
)
cur = conn.cursor()

print("🔍 Carregando embeddings do banco...")
cur.execute("""
    SELECT id, vetor_descricao_questao_testes
      FROM processos
     WHERE vetor_descricao_questao_testes IS NOT NULL
     AND vetor_descricao_questoes_teste_plotagem IS NULL
     ORDER BY id
""")
rows = cur.fetchall()

ids = [r[0] for r in rows]
embeddings = np.vstack([np.array(json.loads(r[1]), dtype=np.float32) for r in rows])
total = len(ids)
print(f"✅ {total} embeddings carregados")

if total == 0:
    cur.close()
    conn.close()
    print("Nada a processar. ✅")
else:
    # =========================
    # Ajusta UMAP (fit) e transforma em lotes de 500
    # =========================
    print("⚙️ Ajustando UMAP (fit único em toda a base)...")
    umap = UMAP(
        random_state=42,
        n_neighbors=15,
        n_components=2,
        min_dist=0.3,
        metric="cosine"
    ) 
    umap.fit(embeddings)
    print("✅ UMAP ajustado")

    TRANSFORM_BATCH_SIZE = 100   # reduz de 500 em 500
    UPDATE_BATCH_SIZE = 100     # salva no banco em lotes (pode ser 500/1000/2000)

    reduced_cache = []  # acumula (embedding_reduzido, id) para updates
    reduzidos = 0       # contador de vetores já reduzidos
    salvos = 0          # contador de vetores já salvos no banco

    update_sql = """
        UPDATE processos
           SET vetor_descricao_questoes_teste_plotagem = %s
         WHERE id = %s
    """

    print("🚚 Iniciando redução em lotes de 500...")
    for start in range(0, total, TRANSFORM_BATCH_SIZE):
        end = min(start + TRANSFORM_BATCH_SIZE, total)
        X_batch = embeddings[start:end]
        Y_batch = umap.transform(X_batch)  # (batch, 5)

        # acumula para atualização no banco
        for y_vec, id_ in zip(Y_batch, ids[start:end]):
            reduced_cache.append((y_vec.tolist(), id_))

        reduzidos += (end - start)

        # feedback
        clear_output(wait=True)
        print(f"⚙️ Reduzindo dimensionalidade com UMAP...")
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")

        # escreve no banco quando atingir UPDATE_BATCH_SIZE (ou no final)
        if len(reduced_cache) >= UPDATE_BATCH_SIZE:
            cur.executemany(update_sql, reduced_cache)
            conn.commit()
            salvos += len(reduced_cache)
            reduced_cache.clear()

            clear_output(wait=True)
            print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
            print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    # salva o restante
    if reduced_cache:
        cur.executemany(update_sql, reduced_cache)
        conn.commit()
        salvos += len(reduced_cache)
        reduced_cache.clear()

        clear_output(wait=True)
        print(f"➡️  {reduzidos} vetores reduzidos do total de {total}")
        print(f"💾 {salvos} vetores salvos no banco (acumulado)")

    print("🎉 Finalizado com sucesso!")
    cur.close()
    conn.close()

➡️  53257 vetores reduzidos do total de 53257
💾 53257 vetores salvos no banco (acumulado)
🎉 Finalizado com sucesso!
